In [1]:
import os
import datetime
from datetime import timedelta
import time
import pandas as pd
import numpy as np
from pandas import DataFrame
from datetime import datetime

time_run = time.time()

In [2]:
xls = pd.ExcelFile("D:\Quynh\PAYROLL\Input file\Costing Examples_NEW.xlsx")
time_df = pd.read_excel(xls, 'Check')
time_df

,Contact Name,Employment Type,Start date,End date,Process Period Type,Process period start date,Process period end date
0,Aaron Edwards,Casual,2021-03-26 22:00:00,2021-03-27 00:30:00,Weekly - Monday to Sunday,2021-03-22,2021-03-28
1,Aaron Edwards,Casual,2021-03-26 22:00:00,2021-03-27 02:00:00,Weekly - Monday to Sunday,2021-03-22,2021-03-28
2,Quynh,Casual,2021-03-27 22:00:00,2021-03-28 07:00:00,Weekly - Monday to Sunday,2021-03-22,2021-03-28
3,Quynh,Full Time,2021-03-26 00:00:00,2021-03-27 00:00:00,Weekly - Monday to Sunday,2021-03-22,2021-03-28
4,Aaron Edwards,Full Time,2021-03-25 06:00:00,2021-03-25 19:30:00,Weekly - Monday to Sunday,2021-03-22,2021-03-28
5,Quynh,Full Time,2021-03-26 06:00:00,2021-03-26 13:30:00,Weekly - Monday to Sunday,2021-03-22,2021-03-28
6,Aaron Edwards,Full Time,2021-04-25 06:00:00,2021-04-25 13:30:00,Weekly - Monday to Sunday,2021-04-22,2021-04-28
7,Quynh,Part Time,2021-04-24 20:00:00,2021-04-25 00:30:00,Weekly - Monday to Sunday,2021-04-22,2021-04-28


In [3]:

holiday_df = xls.parse("Holiday")
holiday_list = [("" if m>9 else "0") + str(m) + "/" + ("" if d>9 else "0") + str(d) for d, m in zip(holiday_df["Day"], holiday_df["Month"])]

base_rate = 20
holiday_rate = 2.5

rate1 = np.array([[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1.5,1.5,1.5,1.5,2],[2,2,2,2,2]])
rate2 = np.array([[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1.5,1.5,1.5,1.5,2],[2,2,2,2,2]])
rate3 = np.array([[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.7,1.7,1.7,1.7,2.2],[2.2,2.2,2.2,2.2,2.2]])   
rate_data = {"Full Time": rate1, "Part Time": rate2, "Casual": rate3}

In [4]:
def is_holiday(date):
    date = str(date).split("-")
    return (date[1] + "/" + date[2]) in holiday_list

def day_val(day):
    val = {"Monday":0,"Tuesday":1,"Wednesday":2,'Thursday':3,"Friday":4,"Saturday":5,"Sunday":6}
    return val[day]

def cal_base_rate(_type, day, date, start_time, end_time, rate_data):
    
    shift = 2;      
    ord_rate = 0
    ot1_rate = 0
    ot2_rate = 0
    
    if is_holiday(date):
        ord_rate = holiday_rate
        ot1_rate = holiday_rate
        ot2_rate = holiday_rate
    else:        
        if _type == "Full Time":
            if start_time < 5 or end_time > 1:
                shift = 2
            elif end_time > 19 or end_time < 1:
                shift = 1
            elif start_time >= 5 and end_time < 20:
                shift = 0
            
        elif _type == "Part Time":
            
            if start_time < 5 or end_time > 1:
                shift = 2
            elif end_time > 19 or end_time < 1:
                shift = 1
            elif start_time >= 5 and end_time < 20:
                shift = 0

        else:
            
            if start_time < 5 or end_time > 1:
                shift = 2
            elif end_time > 19 or end_time < 1:
                shift = 1
            elif start_time >= 5 and end_time < 20:
                shift = 0
            
        ord_rate = rate_data[_type][day_val(day)][shift]
        ot1_rate = rate_data[_type][day_val(day)][3]
        ot2_rate = rate_data[_type][day_val(day)][4]
    return (ord_rate,ot1_rate,ot2_rate)

In [5]:

data= []
for row in time_df.iterrows():    
    _id = row[1]["Contact Name"] 
    _type = row[1]["Employment Type"]
    
    start_period = row[1]['Process period start date'].date()
    end_period = row[1]['Process period end date'].date()
    
    start = row[1]["Start date"]
    end   = row[1]["End date"]   
    start_time = float(start.hour + start.minute/60)
    end_time = float(end.hour + end.minute/60)

    start_date = start.date() 
    end_date   = end.date()
        
    start_day  = start.strftime('%A')
    end_day    = end.strftime('%A') 
    if start_date != end_date and end_time > 0 and ((start_day in ["Friday","Saturday","Sunday"]) or is_holiday(start_date) or is_holiday(end_date)):
        
        rate1 = cal_base_rate(_type, start_day, start_date, start_time, 24      , rate_data)[0]
        rate2 = cal_base_rate(_type, end_day  , end_date  , 0         , end_time, rate_data)[0]
        
        delta_time1 = 24 - start_time   
        delta_time2 = end_time 
        if delta_time1 <3 :
            delta_time1=3
        if delta_time2<3:
            delta_time2=3
            
        date1 = start_date   
        date2= end_date
        
        day1 = start_day
        day2 = end_day       
                
        data.append((_id, start_period, end_period, date1, start_time, 24      , day1, delta_time1, rate1))
        data.append((_id, start_period, end_period, date2, 0         , end_time, day2, delta_time2, rate2))  
     
    else: 
        delta_time = (end-start).total_seconds() / 3600.0
        if delta_time <3:
            delta_time = 3
            
        rate = cal_base_rate(_type, start_day, start_date, start_time, end_time, rate_data)[0]
        ot1_rate = cal_base_rate(_type, start_day, start_date, start_time, end_time, rate_data)[1]
        ot2_rate = cal_base_rate(_type, start_day, start_date, start_time, end_time, rate_data)[2]
        
        data.append((_id,start_period, end_period, start_date, start_time, end_time,start_day, delta_time, rate))
            
file = pd.DataFrame(data,columns=["ID","Start Period", "End period", "Date","Start Hour", "End Hour","Day", "Hour","Rate"])        

file
    

,ID,Start Period,End period,Date,Start Hour,End Hour,Day,Hour,Rate
0,Aaron Edwards,2021-03-22,2021-03-28,2021-03-26,22.0,24.0,Friday,3.0,1.45
1,Aaron Edwards,2021-03-22,2021-03-28,2021-03-27,0.0,0.5,Saturday,3.0,1.70
2,Aaron Edwards,2021-03-22,2021-03-28,2021-03-26,22.0,24.0,Friday,3.0,1.45
3,Aaron Edwards,2021-03-22,2021-03-28,2021-03-27,0.0,2.0,Saturday,3.0,1.70
4,Quynh,2021-03-22,2021-03-28,2021-03-27,22.0,24.0,Saturday,3.0,1.70
5,Quynh,2021-03-22,2021-03-28,2021-03-28,0.0,7.0,Sunday,7.0,2.20
6,Quynh,2021-03-22,2021-03-28,2021-03-26,0.0,0.0,Friday,24.0,1.25
7,Aaron Edwards,2021-03-22,2021-03-28,2021-03-25,6.0,19.5,Thursday,13.5,1.25
8,Quynh,2021-03-22,2021-03-28,2021-03-26,6.0,13.5,Friday,7.5,1.25
9,Aaron Edwards,2021-04-22,2021-04-28,2021-04-25,6.0,13.5,Sunday,7.5,2.50


In [6]:
sum_day = file.groupby(["ID","Start Period", "End period", "Date"]).sum()
sum_day[sum_day.index.get_level_values("Date") == "2021-3-25"]
# sum_day.index
# sum_day.index.names

,,,,Start Hour,End Hour,Hour,Rate
ID,Start Period,End period,Date,,,,


In [7]:
period_pay = {}
for row in file.iterrows():
    _id = row[1]["ID"]
    _period = rowm[1]["Start Period"] 
    _key_period = _id + " " + str(_period)
    _day = row[1]["Start"]
    _hour = row[1]["Hour"]
#     _start = row[1]["Start Hour"]
#     _end = row[1]["End Hour"]
    _rate = row[1]["Rate"]
#     _ot1_rate = row[1]["Rate OT1"]
#     _ot2_rate = row[1]["Rate OT2"]
    days = {}
    
    value = {"days":" ", "sum_hour", " "}
    
#     pay = base_rate*(min([8,_hour])*_rate + max([0,min([2,_hour - 8])])*_ot1_rate + max([0,min([_hour - 10])])*_ot2_rate)
    print(pay)
    if not _key_period in period_pay:
        period_pay[_key_period] = value
    else:
        old_value = period_pay[_key_period]
        days = value["days"]
        
        

SyntaxError: invalid syntax (<ipython-input-7-c73ddaef0b89>, line 15)